# Getting Started with CLassification using TLT 3.0

This notebook will help you quickly pick a model of your choice and initiate training. You can also prune your model, perform INT8 optimization and/or convert to a TensorRT Engine

## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>
The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TLT experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/classification`. More information on how to set up the dataset and the supported steps in the TLT workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [1]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env USER_EXPERIMENT_DIR=/workspace/tlt-experiments/
%env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/classification/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tlt-samples/classification

# Please define this local project directory that needs to be mapped to the TLT docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/classification
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
os.environ["LOCAL_PROJECT_DIR"] = "/home/astr1x/Playground/nvidia-tlt-get-started/"

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "classification/data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "classification"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tlt-experiments/classification/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

env: KEY=nvidia_tlt
env: NUM_GPUS=1
env: USER_EXPERIMENT_DIR=/workspace/tlt-experiments/
env: DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/classification/data
env: SPECS_DIR=/workspace/tlt-experiments/classification/specs
total 8
-rw-rw-r-- 1 astr1x astr1x 1298 Mar  3 15:28 classification_spec.cfg
-rw-rw-r-- 1 astr1x astr1x 1164 Mar  3 15:41 classification_retrain_spec.cfg


The cell below maps the project directory on your local host to a workspace directory in the TLT docker instance, so that the data and the results are mapped from outside to inside of the docker instance.

In [2]:
# Mapping up the local directories to the TLT docker.
import json
import os
mounts_file = os.path.expanduser("~/.tlt_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tlt-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ],
    "DockerOptions":{
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [3]:
# Make sure the output is correct

!cat ~/.tlt_mounts.json

{
    "Mounts": [
        {
            "source": "/home/astr1x/Playground/nvidia-tlt-get-started/",
            "destination": "/workspace/tlt-experiments"
        },
        {
            "source": "/home/astr1x/Playground/nvidia-tlt-get-started/classification/specs",
            "destination": "/workspace/tlt-experiments/classification/specs"
        }
    ],
    "DockerOptions": {
        "user": "1000:1000"
    }
}

## 1. Download pretrained model for transfer learning

We will use ngc to pull the model from the provided list. 

In [4]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_reg_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_reg_linux.zip
--2021-03-03 15:55:23--  https://ngc.nvidia.com/downloads/ngccli_reg_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 99.86.19.112, 99.86.19.49, 99.86.19.74, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|99.86.19.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20111585 (19M) [application/zip]
Saving to: ‘/home/astr1x/Playground/nvidia-tlt-get-started//ngccli/ngccli_reg_linux.zip’

ngccli_reg_linux.zi 100%[===================>]  19.18M  4.13MB/s    in 5.6s    

2021-03-03 15:55:29 (3.41 MB/s) - ‘/home/astr1x/Playground/nvidia-tlt-get-started//ngccli/ngccli_reg_linux.zip’ saved [20111585/20111585]

Archive:  /home/astr1x/Playground/nvidia-tlt-get-started//ngccli/ngccli_reg_linux.zip
  inflating: /home/astr1x/Playground/nvidia-tlt-get-started//ngccli/ngc  
 extracting: /home/astr1x/Playground/nvidia-tlt-get-started//ngccli/ngc.md5  


In [5]:
!ngc registry model list "nvidia/tlt_pretrained_classification:*"

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| vgg19 | 77.56 | 80    | 1     | V100  | 153.7 | 153.7 | UPLOA | Apr   |
|       |       |       |       |       |       | 2 MB  | D_COM | 29,   |
|       |       |       |       |       |       |       | PLETE | 2020  |
| vgg16 | 77.17 | 80    | 1     | V100  | 515.1 | 515.0 | UPLOA | Apr   |
|       |       |       |       |       |       | 9 MB  | D_COM | 29,   |
|       |       |       |       |       |       |       | PLETE | 2020  |
| squee | 65.13 | 80    | 1     | V100  | 6.5   | 6.46  | UPLOA | Apr   |
| zenet |       |       |       |     

Let's go ahead with ResNet18 for classification

In [6]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tlt_pretrained_classification:resnet18 --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

In [7]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/tlt_pretrained_classification_vresnet18

Check that model is downloaded into dir.
total 91100
-rw------- 1 astr1x astr1x 93278448 Mar  3 15:11 resnet_18.hdf5


## 2. Training Specification

You can tune the hyper-parameters and other settings in this specific file.

In [8]:
!cat $LOCAL_SPECS_DIR/classification_spec.cfg

model_config {
  arch: "resnet",
  n_layers: 18
  # Setting these parameters to true to match the template downloaded from NGC.
  use_batch_norm: true
  all_projections: true
  freeze_blocks: 0
  freeze_blocks: 1
  input_image_size: "3,128,128"
}
train_config {
  train_dataset_path: "/workspace/tlt-experiments/classification/data/train"
  val_dataset_path: "/workspace/tlt-experiments/classification/data/val"
  pretrained_model_path: "/workspace/tlt-experiments/classification/pretrained_resnet18/tlt_pretrained_classification_vresnet18/resnet_18.hdf5"
  optimizer {
    sgd {
    lr: 0.01
    decay: 0.0
    momentum: 0.9
    nesterov: False
  }
}
  batch_size_per_gpu: 16
  n_epochs: 2
  n_workers: 4
  preprocess_mode: "caffe"
  enable_random_crop: True
  enable_center_crop: True
  label_smoothing: 0.0
  mixup_alpha: 0.1
  # regularizer
  reg_config {
    type: "L2"
    scope: "Conv2D,Dense"
    weight_decay: 0.00005
  }

  # learning_rate
  lr_config {
    step {
      learning_rate: 0.00

## 3. Start Training

It's time to finally start training!

In [10]:
!tlt classification train -e $SPECS_DIR/classification_spec.cfg -r $USER_EXPERIMENT_DIR/classification/output -k $KEY

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-wewfnkb0 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.




2021-03-03 10:26:14,096 [INFO] __main__: Loading experiment spec at /workspace/tlt-experiments/classification/specs/classification_spec.cfg.

2021-03-03 10:26:14,100 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/scripts/train.py:295: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


2021-03-03 10:26:14,100 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/2


2021-03-03 10:26:35,415 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.


2021-03-03 10:26:35,431 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.


2021-03-03 10:26:37,105 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.


2021-03-03 10:26:37,301 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.


2021-03-03 10:26:38,344 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bi

## 4. Evaluate trained models
Check the accuracy of the trained model


In [11]:
!tlt classification evaluate -e $SPECS_DIR/classification_spec.cfg -k $KEY

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-pluwb_g4 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.



2021-03-03 10:32:24,053 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/scripts/evaluate.py:76: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


2021-03-03 10:32:24,053 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/sc

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 128, 128)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
block_1a_c

Found 5000 images belonging to 2 classes.
2021-03-03 10:32:33,410 [INFO] __main__: Processing dataset (evaluation): /workspace/tlt-experiments/classification/data/val
Evaluation Loss: 0.35510675930976865
Evaluation Top K accuracy: 1.0
Found 5000 images belonging to 2 classes.
2021-03-03 10:32:58,298 [INFO] __main__: Calculating per-class P/R and confusion matrix. It may take a while...
Confusion Matrix
[[2371  129]
 [ 143 2357]]
Classification Report
              precision    recall  f1-score   support

         cat       0.94      0.95      0.95      2500
         dog       0.95      0.94      0.95      2500

   micro avg       0.95      0.95      0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000

2021-03-03 16:03:22,870 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


## 5. Prune Model
Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is depend on the dataset. A pth value 0.68 is just a starting point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.


In [12]:
%env EPOCH=010
!mkdir -p $LOCAL_EXPERIMENT_DIR/output/resnet_pruned
!tlt classification prune -m $USER_EXPERIMENT_DIR/classification/output/weights/resnet_$EPOCH.tlt \
           -o $USER_EXPERIMENT_DIR/classification/output/resnet_pruned/resnet18_nopool_bn_pruned.tlt \
           -eq union \
           -pth 0.5 \
           -k $KEY

env: EPOCH=002
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-obc7h7y6 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.
2021-03-03 10:33:52,639 [INFO] modulus.pruning.pruning: Exploring graph for retainable indices
2021-03-03 10:33:53,442 [INFO] modulus.pruning.pruning: Pruning model and appending pruned nodes to new graph
2021-03-03 10:34:14,329 [INFO] iva.common.magnet_prune: Pruning ratio (pruned model / original model): 0.9860541309430684
2021-03-03 16:04:16,633 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


In [13]:
print('Pruned model:')
print('------------')
!ls -r1t $LOCAL_EXPERIMENT_DIR/output/resnet_pruned

Pruned model:
------------
resnet18_nopool_bn_pruned.tlt


## 6. Retrain the pruned model

In [14]:
!cat $LOCAL_SPECS_DIR/classification_retrain_spec.cfg

model_config {
  arch: "resnet",
  n_layers: 18
  use_batch_norm: true
  all_projections: true
  input_image_size: "3,128,128"
}
train_config {
  train_dataset_path: "/workspace/tlt-experiments/classification/data/train"
  val_dataset_path: "/workspace/tlt-experiments/classification/data/val"
  pretrained_model_path: "/workspace/tlt-experiments/classification/output/resnet_pruned/resnet18_nopool_bn_pruned.tlt"
  optimizer {
    sgd {
    lr: 0.01
    decay: 0.0
    momentum: 0.9
    nesterov: False
  }
}
  batch_size_per_gpu: 16
  n_epochs: 2
  n_workers: 4
  preprocess_mode: "caffe"
  enable_random_crop: True
  enable_center_crop: True
  label_smoothing: 0.0
  mixup_alpha: 0.1
  # regularizer
  reg_config {
    type: "L2"
    scope: "Conv2D,Dense"
    weight_decay: 0.00005
  }

  # learning_rate
  lr_config {
    step {
      learning_rate: 0.006
      step_size: 10
      gamma: 0.1
    }
  }
}
eval_config {
  eval_dataset_path: "/workspace/tlt-experiments/classification/data/val"
  m

In [15]:
!tlt classification train -e $SPECS_DIR/classification_retrain_spec.cfg \
                      -r $USER_EXPERIMENT_DIR/classification/output_retrain \
                      -k $KEY

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-xkloyc8c because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.




2021-03-03 10:34:38,412 [INFO] __main__: Loading experiment spec at /workspace/tlt-experiments/classification/specs/classification_retrain_spec.cfg.

2021-03-03 10:34:38,416 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/scripts/train.py:295: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


2021-03-03 10:34:38,416 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vp


2021-03-03 10:35:06,988 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.


2021-03-03 10:35:07,004 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.


2021-03-03 10:35:08,597 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.


2021-03-03 10:35:08,789 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.


2021-03-03 10:35:09,981 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bi

## 7. Test the model

In [16]:
!tlt classification evaluate -e $SPECS_DIR/classification_retrain_spec.cfg -k $KEY

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-yfqw440v because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.



2021-03-03 10:40:28,318 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/scripts/evaluate.py:76: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


2021-03-03 10:40:28,318 [WARNING] tensorflow: From /home/vpraveen/.cache/dazel/_dazel_vpraveen/216c8b41e526c3295d3b802489ac2034/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/sc

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 128, 128)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 24, 64, 64)   3528        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 24, 64, 64)   96          conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 24, 64, 64)   0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
block_1a_c

Found 5000 images belonging to 2 classes.
2021-03-03 10:40:37,691 [INFO] __main__: Processing dataset (evaluation): /workspace/tlt-experiments/classification/data/val
Evaluation Loss: 0.35500391459465025
Evaluation Top K accuracy: 1.0
Found 5000 images belonging to 2 classes.
2021-03-03 10:41:01,171 [INFO] __main__: Calculating per-class P/R and confusion matrix. It may take a while...
Confusion Matrix
[[2344  156]
 [ 109 2391]]
Classification Report
              precision    recall  f1-score   support

         cat       0.96      0.94      0.95      2500
         dog       0.94      0.96      0.95      2500

   micro avg       0.95      0.95      0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000

2021-03-03 16:11:25,585 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


## 8. Visualize Results!

In [17]:
%env EPOCH=010

!tlt classification inference -e $SPECS_DIR/classification_retrain_spec.cfg \
                          -m $USER_EXPERIMENT_DIR/classification/output_retrain/weights/resnet_$EPOCH.tlt \
                          -k $KEY -b 8 -d $DATA_DOWNLOAD_DIR/test \
                          -cm $USER_EXPERIMENT_DIR/classification/output_retrain/classmap.json

env: EPOCH=002
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-wxlaw8a4 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Using TensorFlow backend.
Using TensorFlow backend.

2021-03-03 10:41:53,147 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


2021-03-03 10:41:53,232 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.


2021-03-03 10:41:53,261 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_gra

2021-03-03 10:42:43,022 [INFO] __main__: Inference complete. Result is saved at /workspace/tlt-experiments/classification/data/test/result.csv
2021-03-03 16:12:45,957 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


In [18]:
!cat $LOCAL_PROJECT_DIR/classification/data/test/result.csv

/workspace/tlt-experiments/classification/data/test/cat.100.jpg,cat,0.9819382429122925
/workspace/tlt-experiments/classification/data/test/cat.10004.jpg,cat,0.9896095991134644
/workspace/tlt-experiments/classification/data/test/cat.10005.jpg,cat,0.8850594162940979
/workspace/tlt-experiments/classification/data/test/cat.10015.jpg,cat,0.9981339573860168
/workspace/tlt-experiments/classification/data/test/cat.10019.jpg,cat,0.9108083844184875
/workspace/tlt-experiments/classification/data/test/cat.10026.jpg,dog,0.5587577819824219
/workspace/tlt-experiments/classification/data/test/cat.10030.jpg,cat,0.9859262704849243
/workspace/tlt-experiments/classification/data/test/cat.10039.jpg,cat,0.9997041821479797
/workspace/tlt-experiments/classification/data/test/cat.10043.jpg,cat,0.7107726335525513
/workspace/tlt-experiments/classification/data/test/cat.10044.jpg,cat,0.929760217666626
/workspace/tlt-experiments/classification/data/test/cat.10045.jpg,cat,0.9984098672866821
/workspace/tlt-experimen

/workspace/tlt-experiments/classification/data/test/dog.4794.jpg,dog,0.9894425868988037
/workspace/tlt-experiments/classification/data/test/dog.4795.jpg,dog,0.7269190549850464
/workspace/tlt-experiments/classification/data/test/dog.4798.jpg,dog,0.9260324239730835
/workspace/tlt-experiments/classification/data/test/dog.48.jpg,dog,0.9987391829490662
/workspace/tlt-experiments/classification/data/test/dog.4816.jpg,dog,0.9788009524345398
/workspace/tlt-experiments/classification/data/test/dog.4823.jpg,dog,0.6312572956085205
/workspace/tlt-experiments/classification/data/test/dog.4824.jpg,dog,0.9972509741783142
/workspace/tlt-experiments/classification/data/test/dog.4826.jpg,dog,0.9997575879096985
/workspace/tlt-experiments/classification/data/test/dog.4830.jpg,dog,0.8542646169662476
/workspace/tlt-experiments/classification/data/test/dog.4835.jpg,dog,0.980550229549408
/workspace/tlt-experiments/classification/data/test/dog.484.jpg,dog,0.9226055145263672
/workspace/tlt-experiments/classific

## 9. Export to ETLT

In [ ]:
!tlt classification export \
            -m $USER_EXPERIMENT_DIR/classification/output_retrain/weights/resnet_$EPOCH.tlt \
            -o $USER_EXPERIMENT_DIR/classification/export/final_model.etlt \
            -k $KEY

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export/